MANQUE LA DISTRIBUTION SELON L'EVOLUTION CHRONOLOGIQUE ( périodes de 10ans) et pour les élections canto/communes

Q4: évolution des candidatures féminines selon les époques et selon les types d'élection cette requête s'intéresse à toutes les candidatures que les femmes on pu effectuer suivant les périodes et les types d'élections. la temporalité s'étend de 1961-1980, avec des période de 10ans (3 périodes) 
besoin :
table personne
>nom/prénom
ID
genre

table participation
>fk_personne
fk_evenement

table événement
>événement
fk_type_E

4.1 en premier lieu, il s'agit de voir de manière globale la participation aux élections législatives

In [ ]:
import sqlite3
import pandas

connection = sqlite3.connect('C:/Users/LZender/Documents/Travail/unine/histoire/mémoire/AEN/grand_conseil/memoire_POP/liste_grand_conseil.db')


cursor = connection.cursor()

In [27]:
annees = list(range(1961, 1982,10))

periodes = []

for index, annee_item in enumerate(annees):
    if index + 1 < len(annees):
        periodes.append({
            'start': annees[index],
            'end': annees[index + 1] - 1
        })

for periode_item in periodes:
    start = periode_item['start']
    end = periode_item['end']
    cursor.execute("""
SELECT COUNT(pk_evenement) FROM personne as pe
LEFT JOIN participation as pa ON pa.fk_personne = pe.pk_personne
LEFT JOIN evenement as e ON e.pk_evenement = pa.fk_evenement
WHERE genre = 'F' AND pa.annee BETWEEN ? AND ?
AND e.definition ='legislatif'
""", [start, end])
    result_femmes = cursor.fetchall()[0][0]
    cursor.execute("""
SELECT COUNT(pk_evenement) FROM personne as pe
LEFT JOIN participation as pa ON pa.fk_personne = pe.pk_personne
LEFT JOIN evenement as e ON e.pk_evenement = pa.fk_evenement
WHERE genre = 'M' AND pa.annee BETWEEN ? AND ?
AND e.definition ='legislatif'
""", [start, end])
    result_hommes = cursor.fetchall()[0][0]
    periode_item['femmes'] = result_femmes
    periode_item['hommes'] = result_hommes


print(periodes)

[{'start': 1961, 'end': 1970, 'femmes': 41, 'hommes': 147}, {'start': 1971, 'end': 1980, 'femmes': 58, 'hommes': 116}]


In [28]:
df = pandas.DataFrame.from_dict(periodes)

df['total'] = df['femmes'] + df['hommes']

df['pourcentage_femmes'] = df['femmes'] / df['total'] * 100.00
df['pourcentage_hommes'] = df['hommes'] / df['total'] * 100.00

df.head()

,start,end,femmes,hommes,total,pourcentage_femmes,pourcentage_hommes
0,1961,1970,41,147,188,21.808511,78.191489
1,1971,1980,58,116,174,33.333333,66.666667


4.1 en premier lieu, il s'agit de voir de manière globale la participation aux élections

4.2 ensuite, l'analyse peut s'intéresser à la participation selon le genre

4.2.1 participation aux élections des militantEs

In [2]:
import sqlite3
import pandas

connection = sqlite3.connect('C:/Users/LZender/Documents/Travail/unine/histoire/mémoire/AEN/grand_conseil/memoire_POP/liste_grand_conseil.db')


cursor = connection.cursor()

df_F = pandas.read_sql("""
SELECT COUNT (pk_evenement) as count_election, e.nom as election FROM personne as pe
LEFT JOIN participation as pa ON pa.fk_personne = pe.pk_personne
LEFT JOIN evenement as e ON e.pk_evenement = pa.fk_evenement

where e.pk_evenement is not null AND pe.genre='F' AND e.definition ='legislatif'
group by pk_evenement
order by count_election DESC
""", connection)

total= df_F['count_election'].sum()

df_F['pourcentage']= df_F['count_election']/ total * 100.00

df_F.head(1000)

,count_election,election,pourcentage
0,64,election_conseil_general,57.657658
1,42,election_parlement,37.837838
2,3,election_conseil_national,2.702703
3,2,election_conseil_des_etats,1.801802


4.2.2 participation aux élections des militants

In [3]:
import sqlite3
import pandas

connection = sqlite3.connect('C:/Users/LZender/Documents/Travail/unine/histoire/mémoire/AEN/grand_conseil/memoire_POP/liste_grand_conseil.db')


cursor = connection.cursor()

df_M = pandas.read_sql("""
SELECT COUNT (pk_evenement) as count_election, e.nom as election FROM personne as pe
LEFT JOIN participation as pa ON pa.fk_personne = pe.pk_personne
LEFT JOIN evenement as e ON e.pk_evenement = pa.fk_evenement

where e.pk_evenement is not null AND pe.genre='M' AND e.definition ='legislatif'
group by pk_evenement
order by count_election DESC
""", connection)

total= df_M['count_election'].sum()

df_M['pourcentage']= df_M['count_election']/ total * 100.00

df_M.head(1000)

,count_election,election,pourcentage
0,175,election_parlement,49.715909
1,169,election_conseil_general,48.011364
2,8,election_conseil_national,2.272727


4.2.3 comparaison des pourcentages entre les genres

In [9]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('C:/Users/LZender/Documents/Travail/unine/histoire/mémoire/AEN/grand_conseil/memoire_POP/liste_grand_conseil.db')

df = pd.read_sql("""
SELECT 
    e.nom as election, 
    pe.genre, 
    COUNT(pk_evenement) AS count_participation
FROM personne AS pe
LEFT JOIN participation AS pa ON pa.fk_personne = pe.pk_personne
LEFT JOIN evenement AS e ON e.pk_evenement = pa.fk_evenement

WHERE e.pk_evenement IS NOT NULL AND e.nom IS NOT 'election_conseil_des_etats' AND e.definition ='legislatif' AND pa.annee BETWEEN 1961 AND 1980
GROUP BY e.pk_evenement, pe.genre
ORDER BY e.pk_evenement, count_participation DESC
""", connection)

df['total_election'] = df.groupby('election')['count_participation'].transform('sum')

df['pourcentage'] = df['count_participation'] / df['total_election'] * 100.0

df[['election', 'genre', 'count_participation', 'pourcentage']]

,election,genre,count_participation,pourcentage
0,election_conseil_national,M,8,72.727273
1,election_conseil_national,F,3,27.272727
2,election_parlement,M,127,75.147929
3,election_parlement,F,42,24.852071
4,election_conseil_general,M,136,70.466321
5,election_conseil_general,F,57,29.533679


,election,genre,count_participation,total_election,pourcentage
0,election_conseil_national,M,8,11,72.727273
1,election_conseil_national,F,3,11,27.272727
2,election_parlement,M,127,169,75.147929
3,election_parlement,F,42,169,24.852071
4,election_conseil_general,M,136,193,70.466321
